In [ ]:
import tensorlayerx as tlx
import tensorlayerx.nn as nn

![](https://sevanthea7.oss-cn-beijing.aliyuncs.com/QGworks/202407311038932.png)

In [ ]:
class Generator( nn.Module ):
    gen_features = 64
    dis_features = 64
    size16 = dis_features // 16                             # 4 -> 特征图的大小
    w_init = nn.initializers.random_normal( 0, 0.02 )       # 权重初始化，使用均值为0，标准差为0.02的正态分布
    g_init = nn.initializers.random_normal( 1., 0.02 )      # 偏置初始化，使用均值为1，标准差为0.02的正态分布
    def __init__( self ):
        super( Generator, self ).__init__()
        self.fc1 = nn.Linear( 
            out_features = self.gen_features * 8 * self.size16 * self.size16, # 512 * 4 * 4
            W_init = self.w_init,
            b_init = None
        )
        self.reshape = nn.Reshape( 
            shape = ( -1, self.size16, self.size16, self.gen_features * 8 )
        )
        
        self.bn1 = nn.BatchNorm2d( 0.9, act = nn.ReLU, gamma_init = self.g_init )
        self.deconv2d1 = nn.ConvTranspose2d( 
            out_channels = self.gen_features * 4,
            kernel_size = ( 5, 5 ),
            stride = ( 2, 2 ),
            W_init = self.w_init,
            b_init = None
        )
        '''
        输入：[ N, 512, 4, 4 ]
        输出高度：( 4 - 1 ) * 2 + 5 = 11
        输出宽度：( 4 - 1 ) * 2 + 5 = 11
        -> 输出[ N, 256, 11, 11 ]
        '''
        
        
        self.bn2 = nn.BatchNorm2d( 0.9, act = nn.ReLU, gamma_init = self.g_init )
        self.deconv2d2 = nn.ConvTranspose2d(
            out_channels = self.gen_features * 2,
            kernel_size = ( 5, 5 ),
            stride = ( 2, 2 ), 
            W_init = self.w_init,
            b_init = None
        )
        '''
        [ N, 256, 11, 11 ]
        ( 11 - 1 ) * 2 + 5 = 25
        -> [ N, 128, 25, 25 ]
        '''
        
        self.bn3 = nn.BatchNorm2d( 0.9, act = nn.ReLU, gamma_init = self.g_init )
        self.deconv2d3 = nn.ConvTranspose2d(
            out_channels = self.gen_features,
            kernel_size = ( 5, 5 ),
            stride = ( 2, 2 ),
            W_init = self.w_init,
            b_init = None
        )
        '''
        [ N, 128, 25, 25 ]
        ( 25 - 1 ) * 2 + 5 = 53
        -> [ N, 64, 53, 53 ]
        '''
        
        self.bn4 = nn.BatchNorm2d( 0.9, act = nn.ReLU, gamma_init = self.g_init )
        self.deconv2d4 = nn.ConvTranspose2d(
            out_channels = 3,
            kernel_size = ( 5, 5 ),
            stride = ( 2, 2 ),
            W_init = self.w_init,
            b_init = None
        )
        '''
        [ N, 64, 53, 53 ]
        ( 53 - 1 ) * 2 + 5 = 109
        -> [ N, 3, 109, 109 ]
        '''
    
    def forward( self, x ):
        x = self.fc1( x )
        x = self.reshape( x )
        
        x = self.bn1( x )
        x = self.deconv2d1( x )
        x = self.bn2( x )
        x = self.deconv2d2( x )
        x = self.bn3( x )
        x = self.deconv2d3( x )
        x = self.bn4( x )
        x = self.deconv2d4( x )
        
        return x

反卷积公式
$$
H_{out} = (H_{in} - 1) \times \text{stride}[0] - 2 \times \text{padding}[0] + \text{kernel_size}[0] + \text{output\_padding}[0]\\
W_{out} = (W_{in} - 1) \times \text{stride}[1] - 2 \times \text{padding}[1] + \text{kernel_size}[1] + \text{output\_padding}[1]
$$

In [ ]:
class Discriminator( nn.Module ):
    dis_features = 64
    w_init = nn.initializers.random_normal( 0.02 )
    g_init = nn.initializers.random_normal( 1., 0.02 )
    
    def __init__( self ):
        super( Discriminator, self ).__init__()
        self.conv1 = nn.Conv2d( 
            out_channels = self.dis_features,
            kernel_size = ( 5, 5 ),
            stride = ( 2, 2 ),
            act = nn.LeakyReLU( 0.2 ),
            W_init = self.w_init
        )
        self.conv2 = nn.Conv2d(
            out_channels = self.dis_features * 2,
            kernel_size = ( 5, 5 ),
            stride = ( 2, 2 ),
            W_init = self.w_init,
            b_init = None
        )
        self.bn1 = nn.BatchNorm2d( 0.9, act = nn.LeakyReLU( 0.2 ), gamma_init = self.g_init )
        
        self.conv3 = nn.Conv2d( 
            out_channels = self.dis_features * 4,
            kernel_size = ( 5, 5 ),
            stride = ( 2, 2 ),
            W_init = self.w_init,
            b_init = None
        )
        self.bn2 = nn.BatchNorm2d( 0.9, act = nn.LeakyReLU( 0.2 ), gamma_init = self.g_init )
        
        self.conv4 = nn.Conv2d( 
            out_channels = self.dis_features * 8,
            kernel_size = ( 5, 5 ),
            stride = ( 2, 2 ),
            W_init = self.w_init,
            b_init = None
        )
        self.bn3 = nn.BatchNorm2d( 0.9, act = nn.LeakyReLU( 0.2 ), gamma_init = self.g_init )
        
        self.flatten = nn.Flatten()
        self.fc = nn.Linear( 1, self.w_init )
        
    
    def forward( self, x ):
        x = self.conv1( x )
        x = self.conv2( x )
        x = self.bn1( x )
        x = self.conv3( x )
        x = self.bn2( x )
        x = self.conv4( x )
        x = self.bn3( x )
        
        x = self.flatten( x )
        x = self.fc( x )
        
        return x 

数据集

In [ ]:
import numpy as np
from tensorboardX import transforems, load_images

训练

In [ ]:
class Dis_Loss( nn.Module ):
    def __init__( self, Dis, Gen ):
        super( Dis_Loss, self ).__init__()
        self.Dis = Dis
        self.Gen = Gen
    
    def forward( self, images, fake ):
        dis_logits_f = self.Dis( self.Gen( fake ) )
        dis_logits_r = self.Dis( images )
        dis_loss_real = tlx.losses.sigmoid_cross_entropy( dis_logits_r, tlx.ones_like( dis_logits_r ) )
        dis_loss_fake = tlx.losses.sigmoid_cross_entropy( dis_logits_f, tlx.zeros_like( dis_logits_f ) )
        dis_loss = dis_loss_real + dis_loss_fake
        return dis_loss
        

In [ ]:
class Gen_Loss( nn.Module ):
    def __init__( self, Dis, Gen ):
        super( Gen_Loss, self ).__init__()
        self.Dis = Dis
        self.Gen = Gen
        
    def forward( self, images, fake ):
        dis_logits = self.Dis( self.Gen( fake ) )
        gen_loss = tlx.losses.sigmoid_cross_entropy( dis_logits, tlx.ones_like( dis_logits ) )
        return gen_loss

In [ ]:
num_epoch = 25
num_noise = 100
num_batch = 32
num_output = 64
sample_size = 64
num_tiles = int( np.sqrt( sample_size ) )

In [ ]:
from tensorlayerx.model import TrainOneStep
from tensorlayerx.utils.visualize import save_images
from MyData import get_celebA
def train():
    
    images_loader, images_path = get_celebA( num_batch )              #从自定义的的数据集导入数据
    
    
    Gen = Generator()
    Dis = Discriminator()
    Gen.init_build( nn.Input( shape = ( num_batch, 100 ) ) )
    Dis.init_build( nn.Input( shape = ( num_batch, 64, 64, 3 ) ) )
    
    Gen.set_train()
    Dis.set_train()
    
    dis_optimizer = tlx.optimizers.Adam( lr = 0.0002, beta_1 = 0.5 )
    gen_optimizer = tlx.optimizers.Adam( lr = 0.0002, beta_1 = 0.5 )
    
    g_w = Gen.trainable_weights()
    d_w = Dis.trainable_weights()
    
    net_loss_Dis = Dis_Loss( Gen, Dis )
    net_loss_Gen = Gen_Loss( Gen, Dis )
    
    train_Gen = TrainOneStep( net_loss_Gen, optimizer = gen_optimizer, train_weights = g_w )
    train_Dis = TrainOneStep( net_loss_Dis, optimizer = dis_optimizer, train_weights = d_w )
    
    for epoch in range( num_epoch ):
        for i, batch in enumerate( images_loader ):
            # z -> noise 0到1之间的随机噪声
            z = np.random.normal( loc = 0.0, scale = 1.0, size = [ num_batch, num_noise ] )
            z = tlx.ops.convert_to_tensor( z )
            d_loss = train_Dis( batch, z )
            g_loss = train_Gen( batch, z )
            
        Gen.save_weights( '../data/Gen.npz', format = 'npz' )
        Dis.save_weights( '../data/Dis.npz', format = 'npz' )
        
        Gen.set_eval()
        result = Gen( z )
        Gen.set_train()
        save_images( 
            images = tlx.convert_to_numpy( result ),
            size = [ num_tiles, num_tiles ],
            image_path = f'../data/train_{epoch}.png'
        )